In [1]:

from keras import backend as K
from keras.models import load_model
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import cv2
import time

import pickle
# for scaling and inverse_transform
from sklearn.preprocessing import MinMaxScaler

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
model_path = "30f_future-model.h5"
model = load_model(model_path)
n_seq = 30
n_seq_future = 30

In [3]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

In [4]:
jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')

In [5]:
from jaad_data import JAAD

In [6]:
imdb = JAAD(data_path = jaad_path)

In [7]:
def play():
    global batch
    img_width, img_height = 960, 540
    start_time_video = time.time()
    clip_path = "/home/kara9147/JAAD/JAAD_clips/" + vid + ".mp4"
    cap = cv2.VideoCapture(clip_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print( length, frames[0] )
    
    # Time to read all frames, predict and put bounding boxes around them, and show them.
    i = 0
    total_time = 0.0
    pred = True

    # Capture frame-by-frame
    ret = True
    
    while(ret and i <  frames[0] + n_seq):
        ret, origimg = cap.read()
        i += 1
    print(i)
    i = 0
    while(ret and i < lstm_seq.shape[0] - (n_seq + n_seq_future) -1):
        ret, origimg = cap.read()
        batch = lstm_seq[i:i + n_seq]
        print(batch)
       
        i = i + 1
        #print("Processing {} th frame".format(i))
        if (ret != False ):
            #print(img.shape)
            img = cv2.resize(origimg, (img_width, img_height))
            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            print(y_hat)

            #batch =  np.delete(batch, 0, axis = 0)
            #batch =  np.append(batch, y_hat, axis = 0)
            #print(len(batch))
            
            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # 4: Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            print(inv_yhat)
            #inv_yhat = y_hat[0]
           
            np.set_printoptions(precision=2, suppress=True, linewidth=90)
            # print("Predicted boxes:\n")
            # print('   class   conf xmin   ymin   xmax   ymax')

            #print(y_pred_decoded)

            #print(time.time() - start_time)

            if pred:
                xmin = int(inv_yhat[0] / 2)
                ymin = int(inv_yhat[1] / 2)
                xmax = int(inv_yhat[2] / 2)
                ymax = int(inv_yhat[3] / 2)
            else:
                xmin = int(bb_cross[ped_b[ped_index]][i][0] / 2)
                ymin = int(bb_cross[ped_b[ped_index]][i][1] / 2)
                xmax = int(bb_cross[ped_b[ped_index]][i][2] / 2)
                ymax = int(bb_cross[ped_b[ped_index]][i][3] / 2)

            print((xmin, ymin))
            print((xmax, ymax))
            # 1920x1080
            #cv2.rectangle(img, (0,  0 ), (1910, 1070), (255, 0, 0), 2)

            # Draw the predicted boxes in blue
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax ), (255, 0, 0), 1)

            #xmin_ = int (df[i:i+1][0] / 2)
            #ymin_ = int (df[i:i+1][1] / 2)
            #xmax_ = int (df[i:i+1][2] / 2)
            #ymax_ = int (df[i:i+1][3] / 2)
            
            xmin_ = int (df[i+n_seq -1 : i+n_seq][0] / 2)
            ymin_ = int (df[i+n_seq -1 : i+n_seq][1] / 2)
            xmax_ = int (df[i+n_seq -1 : i+n_seq][2] / 2)
            ymax_ = int (df[i+n_seq -1 : i+n_seq][3] / 2)
            
            # Draw the present boxes in green
            cv2.rectangle(img, (xmin_, ymin_), (xmax_, ymax_ ), (0, 255, 0), 1)
            
            # Draw the actual future boxes in red
            cv2.rectangle(img, (int(df[i+(n_seq + n_seq_future) -1 : i + (n_seq + n_seq_future)][0] / 2),
                          int(df[i+(n_seq + n_seq_future) -1 : i+ (n_seq + n_seq_future)][1] / 2)),
                          (int(df[i+(n_seq + n_seq_future) -1 : i+ (n_seq + n_seq_future)][2] / 2),
                          int(df[i+(n_seq + n_seq_future) -1 : i+ (n_seq + n_seq_future)][3] / 2))
                          , (0, 0,255), 1)


            # Display the resulting frame
            cv2.imshow('frame',img)

            # waitKey: 0, wait indefinitely
            if cv2.waitKey(33) & 0xFF == ord('q'):
                break

    end_time_video = time.time()
    print("No of frames: {}".format(i))
    print("Total Time: {}".format(total_time))
    print("fps: {}".format(i / (total_time)))

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [106]:
vid = "video_0309"

In [107]:
anno = imdb._get_annotations(vid)

In [108]:
anno['width']

1920

In [109]:
bb_cross={}
ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]

for i, p in enumerate(ped_b):
    #if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
        bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
        print(p)
print(ped_b)
#for k,v in bb_cross.items():
#    print(k)

0_309_2437b
0_309_2441b
0_309_2439b
0_309_2434b
0_309_2436b
0_309_2435b
0_309_2440b
0_309_2438b
['0_309_2437b', '0_309_2441b', '0_309_2439b', '0_309_2434b', '0_309_2436b', '0_309_2435b', '0_309_2440b', '0_309_2438b']


In [110]:
ped_index = 0
lstm_seq =  bb_cross[ped_b[ped_index]]
frames = (anno['ped_annotations'][ped_b[ped_index]]['frames'][0], len(anno['ped_annotations'][ped_b[ped_index]]['frames']))

In [111]:
frames
#len(lstm_seq)
#l = [k for k,v in anno['ped_annotations'][ped_b[ped_index]].items()]

(0, 20)

In [112]:
df = pd.DataFrame(lstm_seq)
df = df.astype('float32')

for i in range(df.shape[0]):
    xmin_ = int (df[i:i+1][0] / 2)
    ymin_ = int (df[i:i+1][1] / 2)
    xmax_ = int (df[i:i+1][2] / 2)
    ymax_ = int (df[i:i+1][3] / 2)
    #print(i, xmin_, ymin_, xmax_, ymax_)

In [113]:
#pickle_in = open("min-max-scaler.pkl","rb")
#scaler = pickle.load(pickle_in)
#print(type(scaler))
#print(scaler.data_max_)
#print(scaler.data_min_)
scaler = MinMaxScaler(feature_range=(0, 1))
lstm_seq = scaler.fit_transform(lstm_seq)
#batch = scaler.transform(df)

In [114]:
#lstm_seq.shape[0]
#frames[1]

In [115]:
play()

90 0
30
No of frames: 0
Total Time: 0.0


ZeroDivisionError: float division by zero